In [4]:
import os
import numpy as np
import random
import pathlib

class Shuffler:
    def __init__(self, array: np.ndarray) -> None:
        self.original = array
        self.shuffled = self.original.copy()
        self.x = self.original.shape[1]
        self.y = self.original.shape[0]
        self._pieces = []

    def _split(self, x: int, y: int, x_list: list, y_list: list) -> None:
        if len(self._pieces) > 0:
            self._pieces.clear()

        for x_n, x_piece in enumerate(x_list):
            for y_n, y_piece in enumerate(y_list):
                self._pieces.append(
                    self.original[y_n * y:y_piece, x_n * x:x_piece]
                )

    def _generate_image(self, cols: int) -> None:
        chunks = [
            np.vstack(chunk) for chunk in zip(*[iter(self._pieces)] * cols)
        ]
        self.shuffled = np.hstack(np.array(chunks, dtype=np.uint8))

    def shuffle(self, matrix: tuple) -> np.ndarray:
        print(f'Original shape: {self.original.shape}')
        
        x = int(self.x / matrix[0])
        x_list = list(range(x, self.x + 1, x))

        y = int(self.y / matrix[1])
        y_list = list(range(y, self.y + 1, y))

        self._split(x, y, x_list, y_list)
        random.shuffle(self._pieces)

        self._generate_image(matrix[1])
        
        print(f'Shuffled shape: {self.shuffled.shape}')
        return self.shuffled

    def split_and_shuffle(self, t: int, direction: str) -> np.ndarray:
        print(f'Original shape: {self.original.shape}')
        
        if direction == 'h':
            y = int(self.y / (t + 1))
            y_list = list(range(y, self.y + 1, y))
            x_list = [self.x]
            self._split(self.x, y, x_list, y_list)
        elif direction == 'v':
            x = int(self.x / (t + 1))
            x_list = list(range(x, self.x + 1, x))
            y_list = [self.y]
            self._split(x, self.y, x_list, y_list)
        else:
            raise ValueError("Direction must be 'h' for horizontal or 'v' for vertical")

        # Keep one part in the original position and rotate the others by 180n degrees
        keep_index = random.randint(0, t)
        for i in range(len(self._pieces)):
            if i != keep_index:
                n = random.choice([1,2])
                self._pieces[i] = np.rot90(self._pieces[i], k=2*n)

        if direction == 'h':
            self.shuffled = np.vstack(self._pieces)
        elif direction == 'v':
            self.shuffled = np.hstack(self._pieces)

        print(f'Shuffled and rotated shape: {self.shuffled.shape}')
        return self.shuffled

def shuffle_npy_files(src_folder: str, dst_folder: str, matrix: tuple, split_params: tuple = None) -> None:
    total_files = sum([len(files) for r, d, files in os.walk(src_folder) if any(f.endswith('.npy') for f in files)])
    processed_files = 0

    for root, _, files in os.walk(src_folder):
        for file in files:
            if file.endswith('.npy'):
                src_file_path = os.path.join(root, file)
                dst_file_path = os.path.join(
                    dst_folder,
                    os.path.relpath(root, src_folder),
                    file
                )
                os.makedirs(os.path.dirname(dst_file_path), exist_ok=True)

                array = np.load(src_file_path)
                shuffler = Shuffler(array)
                if split_params:
                    shuffled_array = shuffler.split_and_shuffle(*split_params)
                else:
                    shuffled_array = shuffler.shuffle(matrix)
                np.save(dst_file_path, shuffled_array)

                processed_files += 1
                print(f'Processed {processed_files}/{total_files} files')

src_folder = 'data/top_5_compressed'
dst_folder = 'data_shuffle/top_5_compressed'
matrix = (4, 4)
split_params = (1, 'h')  # Example: split into 2 equal parts horizontally and shuffle

shuffle_npy_files(src_folder, dst_folder, matrix, split_params)

Original shape: (224, 224, 6)
Shuffled and rotated shape: (224, 224, 6)
Processed 1/1100 files
Original shape: (96, 5)
Shuffled and rotated shape: (96, 5)
Processed 2/1100 files
Original shape: (224, 224, 6)
Shuffled and rotated shape: (224, 224, 6)
Processed 3/1100 files
Original shape: (65, 5)
Shuffled and rotated shape: (64, 5)
Processed 4/1100 files
Original shape: (224, 224, 6)
Shuffled and rotated shape: (224, 224, 6)
Processed 5/1100 files
Original shape: (224, 224, 3)
Shuffled and rotated shape: (224, 224, 3)
Processed 6/1100 files
Original shape: (224, 224)
Shuffled and rotated shape: (224, 224)
Processed 7/1100 files
Original shape: (224, 224, 3)
Shuffled and rotated shape: (224, 224, 3)
Processed 8/1100 files
Original shape: (121, 5)
Shuffled and rotated shape: (120, 5)
Processed 9/1100 files
Original shape: (224, 224, 6)
Shuffled and rotated shape: (224, 224, 6)
Processed 10/1100 files
Original shape: (99, 5)
Shuffled and rotated shape: (98, 5)
Processed 11/1100 files
Origi

In [5]:
import numpy as np
import cv2 as cv
import os

def npy_to_png(npy_file: str, png_file: str) -> None:
    # Load the .npy file
    array = np.load(npy_file)
    
    # Select the first three channels
    if array.shape[2] >= 3:
        array = array[:, :, :3]
    else:
        raise ValueError("The array does not have enough channels to select 3.")

    # Normalize the array to the range 0-255
    normalized_array = cv.normalize(array, None, 0, 255, cv.NORM_MINMAX)
    
    # Convert to uint8
    uint8_array = normalized_array.astype(np.uint8)
    
    # Ensure the directory exists
    #os.makedirs(os.path.dirname(png_file), exist_ok=True)
    
    # Remove the file if it exists
    if os.path.exists(png_file):
        os.remove(png_file)
    
    # Save as .png file
    cv.imwrite(png_file, uint8_array)
    print(f'Saved {png_file}')





In [6]:
#npy_file = 'data/top_5_compressed/test/figurines/13ec920416ec5b7786d06929bacb135d/0_13ec920416ec5b7786d06929bacb135d_RGB.npy'
npy_file = 'data_shuffle/top_5_compressed/test/figurines/13ec920416ec5b7786d06929bacb135d/0_13ec920416ec5b7786d06929bacb135d_RGB.npy'
png_file = 'shuffle_figurine_RGB.png'
npy_to_png(npy_file, png_file)

Saved shuffle_figurine_RGB.png


In [7]:
#npy_file = 'data/top_5_compressed/test/Lamp/12aa829de18f4e6ad5cb6d178687b980/0_12aa829de18f4e6ad5cb6d178687b980_RGB.npy'
npy_file = 'data_shuffle/top_5_compressed/test/Lamp/12aa829de18f4e6ad5cb6d178687b980/0_12aa829de18f4e6ad5cb6d178687b980_RGB.npy'
png_file = 'shuffle_lamp_RGB.png'
npy_to_png(npy_file, png_file)

Saved shuffle_lamp_RGB.png


ModuleNotFoundError: No module named 'multi_task_models'